In [1]:
import sys
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
import re

In [2]:
dataset = 'LATM2013'

print('Loading raw data for', dataset, '...')
data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/c3mb25524g_7.csv', header=0)
print('Raw data loaded.')
data

Loading raw data for LATM2013 ...
Raw data loaded.


,Protein Group Accessions,# of Protein Groups,Protein Descriptions,Acc. # 1,Sites \nAcc. # 1,Protein Names \nAcc. # 1,Gene Names\nAcc. # 1,Phosposite Sequence Context,Ratio (Medium/Light) Forward,Ratio (Light/Medium) Reverse,...,Activation Type Reverse,Fraction # Identified Reverse,# of Missed Cleavages Reverse,MH+ [Da] Reverse,Charge Reverse,m/z [Da] Reverse,ΔM [ppm] Reverse,RT [min] Reverse,Scan # Reverse,Spectrum File Reverse
0,A0JLT2,1,Mediator of RNA polymerase II transcription su...,A0JLT2,S194,Mediator of RNA polymerase II transcription su...,MED19 LCMR1,PVPPETPSDSDHKKK,1.13,1.59,...,ETD,27,0,1885.82187,3,629.27881,0.14,48.31,3319,20110105_LM_HJ_Epac_Rev_007_F27_2.raw
1,A0JNW5,1,UHRF1-binding protein 1-like OS=Homo sapiens G...,A0JNW5,S989,UHRF1-binding protein 1-like,UHRF1BP1L KIAA0701,GLVYKSGSGEIGSET,0.98,1.04,...,HCD,28,1,1545.70244,2,773.35486,-1.30,38.54,2799,20110114_LM_HJ_Epac_Rev_007_F28.raw
2,A0MZ66,1,Shootin-1 OS=Homo sapiens GN=KIAA1598 PE=1 SV=...,A0MZ66,S506,Shootin-1,KIAA1598,LATSESKSMPVLGSV,1.33,1.41,...,CID,13,0,1451.76702,2,726.38715,0.19,74.89,5576,20110105_LM_HJ_Epac_Rev_007_F13_Multistage.raw
3,A1A4S6,1,Rho GTPase-activating protein 10 OS=Homo sapie...,A1A4S6,S591,Rho GTPase-activating protein 10 (GTPase regul...,ARHGAP10 GRAF2,EPTCLSASPPNAPPR,1.16,3.41,...,CID,14,0,2558.15842,3,853.39099,-1.95,108.31,8630,20110105_LM_HJ_Epac_Rev_007_F14_110107121534.raw
4,A1L170,1,Uncharacterized protein FLJ13137 OS=Homo sapie...,A1L170,S223,Uncharacterized protein C1orf226,C1orf226,TECRRASSPSLIERN,1.06,0.98,...,CID,14,0,1067.51164,2,534.25946,-1.56,50.90,3847,20110105_LM_HJ_Epac_Rev_007_F14_110107121534.raw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6581,Q9Y6R4,1,Mitogen-activated protein kinase kinase kinase...,Q9Y6R4,S1251,Mitogen-activated protein kinase kinase kinase...,MAP3K4 KIAA0213 MAPKKK4 MEKK4 MTK1,RSLSRHSSPTEERDE,0.92,1.27,...,ETD,32,1,1878.80252,3,626.93903,0.35,41.54,2850,20110105_LM_HJ_Epac_Rev_007_F32.raw
6582,Q9Y6R4,1,Mitogen-activated protein kinase kinase kinase...,Q9Y6R4,S1273,Mitogen-activated protein kinase kinase kinase...,MAP3K4 KIAA0213 MAPKKK4 MEKK4 MTK1,SSGSTRRSWELRTLI,0.99,1.55,...,ETD,34,2,1701.77048,3,567.92834,-0.02,67.06,4394,20110105_LM_HJ_Epac_Rev_007_F34.raw
6583,Q9Y6W5;Q8IV90,2,Wiskott-Aldrich syndrome protein family member...,Q9Y6W5,S103,Wiskott-Aldrich syndrome protein family member...,WASF2 WAVE2,TRKAFRSSTIQDQKL,0.81,0.97,...,HCD,29,1,1416.68682,2,708.84705,-1.06,54.27,3883,20110105_LM_HJ_Epac_Rev_007_F29_2.raw
6584,Q9Y6X9,1,MORC family CW-type zinc finger protein 2 OS=H...,Q9Y6X9,S739,MORC family CW-type zinc finger protein 2 (Zin...,MORC2 KIAA0852 ZCWCC1,ATPSRKRSVAVSDEE,1.20,1.69,...,HCD,30,2,2306.97928,3,769.66461,0.11,93.82,7687,20110114_LM_HJ_Epac_Rev_007_F30.raw


In [3]:
data.columns = data.columns.str.strip()

In [4]:
data['Amino acid'] = data['Sites \nAcc. # 1'].str.extract(r'([A-Z])')        # Amino acid
data['Position'] = data['Sites \nAcc. # 1'].str.extract(r'(\d+)')    # Position
print(data[['Sites \nAcc. # 1', 'Amino acid', 'Position']].head())


  Sites \nAcc. # 1 Amino acid Position
0             S194          S      194
1             S989          S      989
2             S506          S      506
3             S591          S      591
4             S223          S      223


In [5]:
data = data.rename(columns={'Gene Names\nAcc. # 1': 'Gene Names'})

In [6]:
# Filtering out semi-colons from 'Amino acid', 'Positions within proteins', and 'Gene names' columns
data = data[~data['Amino acid'].str.contains(';', na=False)]
data = data[~data['Position'].str.contains(';', na=False)]
data = data[~data['Gene Names'].str.contains(';', na=False)]
data = data[data['Gene Names'].str.strip().str.count(r'\s') == 0]
data = data[~data['Gene Names'].str.strip().eq('Deleted')]
data

,Protein Group Accessions,# of Protein Groups,Protein Descriptions,Acc. # 1,Sites \nAcc. # 1,Protein Names \nAcc. # 1,Gene Names,Phosposite Sequence Context,Ratio (Medium/Light) Forward,Ratio (Light/Medium) Reverse,...,# of Missed Cleavages Reverse,MH+ [Da] Reverse,Charge Reverse,m/z [Da] Reverse,ΔM [ppm] Reverse,RT [min] Reverse,Scan # Reverse,Spectrum File Reverse,Amino acid,Position
2,A0MZ66,1,Shootin-1 OS=Homo sapiens GN=KIAA1598 PE=1 SV=...,A0MZ66,S506,Shootin-1,KIAA1598,LATSESKSMPVLGSV,1.33,1.41,...,0,1451.76702,2,726.38715,0.19,74.89,5576,20110105_LM_HJ_Epac_Rev_007_F13_Multistage.raw,S,506
4,A1L170,1,Uncharacterized protein FLJ13137 OS=Homo sapie...,A1L170,S223,Uncharacterized protein C1orf226,C1orf226,TECRRASSPSLIERN,1.06,0.98,...,0,1067.51164,2,534.25946,-1.56,50.90,3847,20110105_LM_HJ_Epac_Rev_007_F14_110107121534.raw,S,223
11,A6ND36,1,Protein FAM83G OS=Homo sapiens GN=FAM83G PE=1 ...,A6ND36,S760,Protein FAM83G,FAM83G,RLLPDPGSPRLAQNA,0.44,0.58,...,0,1063.54802,2,532.27765,-0.54,80.31,6129,20110105_LM_HJ_Epac_Rev_007_F14.raw,S,760
12,A6NFI3,1,Zinc finger protein 316 OS=Homo sapiens GN=ZNF...,A6NFI3,S112,Zinc finger protein 316,ZNF316,SRGGDAKSPVLQEKG,1.03,1.03,...,1,1404.78504,3,468.93320,-2.69,38.71,2812,20110105_LM_HJ_Epac_Rev_007_F28.raw,S,112
14,A6NKD9,1,Coiled-coil domain-containing protein 85C OS=H...,A6NKD9,S246,Coiled-coil domain-containing protein 85C,CCDC85C,KAGATRRSLDDLSAP,1.04,1.23,...,0,1456.68913,2,728.84821,0.59,87.46,7311,20110117_LM_HJ_Epac_Rev_007_F33_2.raw,S,246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6560,Q9Y6I3,1,Epsin-1 OS=Homo sapiens GN=EPN1 PE=1 SV=1 - [E...,Q9Y6I3,T435,Epsin-1 (EH domain-binding mitotic phosphoprot...,EPN1,GSPPPAATPTPTPPT,0.22,0.43,...,0,2364.12320,3,788.71259,1.90,67.74,5160,20110105_LM_HJ_Epac_Rev_007_F9.raw,T,435
6561,Q9Y6I3,1,Epsin-1 OS=Homo sapiens GN=EPN1 PE=1 SV=1 - [E...,Q9Y6I3,T439,Epsin-1 (EH domain-binding mitotic phosphoprot...,EPN1,PAATPTPTPPTRKTP,0.22,0.43,...,0,2364.12320,3,788.71259,1.90,67.74,5160,20110105_LM_HJ_Epac_Rev_007_F9.raw,T,439
6564,Q9Y6K1,1,DNA (cytosine-5)-methyltransferase 3A OS=Homo ...,Q9Y6K1,S105,DNA (cytosine-5)-methyltransferase 3A (Dnmt3a)...,DNMT3A,EPQPEEGSPAGGQKG,1.00,1.12,...,1,1789.81108,2,895.40918,-0.36,26.93,2058,20110105_LM_HJ_Epac_Rev_007_F29_2.raw,S,105
6578,Q9Y6R0,1,Numb-like protein OS=Homo sapiens GN=NUMBL PE=...,Q9Y6R0,S224,Numb-like protein (Numb-related protein) (Numb-R),NUMBL,TSFAREGSFRLSGGG,0.82,0.96,...,1,1683.79465,3,561.93640,-1.01,65.83,4199,20110105_LM_HJ_Epac_Rev_007_F33.raw,S,224


In [7]:
# filter data
data['Phosposite Sequence Context'] = data['Phosposite Sequence Context'].str.replace('_', '')
data

,Protein Group Accessions,# of Protein Groups,Protein Descriptions,Acc. # 1,Sites \nAcc. # 1,Protein Names \nAcc. # 1,Gene Names,Phosposite Sequence Context,Ratio (Medium/Light) Forward,Ratio (Light/Medium) Reverse,...,# of Missed Cleavages Reverse,MH+ [Da] Reverse,Charge Reverse,m/z [Da] Reverse,ΔM [ppm] Reverse,RT [min] Reverse,Scan # Reverse,Spectrum File Reverse,Amino acid,Position
2,A0MZ66,1,Shootin-1 OS=Homo sapiens GN=KIAA1598 PE=1 SV=...,A0MZ66,S506,Shootin-1,KIAA1598,LATSESKSMPVLGSV,1.33,1.41,...,0,1451.76702,2,726.38715,0.19,74.89,5576,20110105_LM_HJ_Epac_Rev_007_F13_Multistage.raw,S,506
4,A1L170,1,Uncharacterized protein FLJ13137 OS=Homo sapie...,A1L170,S223,Uncharacterized protein C1orf226,C1orf226,TECRRASSPSLIERN,1.06,0.98,...,0,1067.51164,2,534.25946,-1.56,50.90,3847,20110105_LM_HJ_Epac_Rev_007_F14_110107121534.raw,S,223
11,A6ND36,1,Protein FAM83G OS=Homo sapiens GN=FAM83G PE=1 ...,A6ND36,S760,Protein FAM83G,FAM83G,RLLPDPGSPRLAQNA,0.44,0.58,...,0,1063.54802,2,532.27765,-0.54,80.31,6129,20110105_LM_HJ_Epac_Rev_007_F14.raw,S,760
12,A6NFI3,1,Zinc finger protein 316 OS=Homo sapiens GN=ZNF...,A6NFI3,S112,Zinc finger protein 316,ZNF316,SRGGDAKSPVLQEKG,1.03,1.03,...,1,1404.78504,3,468.93320,-2.69,38.71,2812,20110105_LM_HJ_Epac_Rev_007_F28.raw,S,112
14,A6NKD9,1,Coiled-coil domain-containing protein 85C OS=H...,A6NKD9,S246,Coiled-coil domain-containing protein 85C,CCDC85C,KAGATRRSLDDLSAP,1.04,1.23,...,0,1456.68913,2,728.84821,0.59,87.46,7311,20110117_LM_HJ_Epac_Rev_007_F33_2.raw,S,246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6560,Q9Y6I3,1,Epsin-1 OS=Homo sapiens GN=EPN1 PE=1 SV=1 - [E...,Q9Y6I3,T435,Epsin-1 (EH domain-binding mitotic phosphoprot...,EPN1,GSPPPAATPTPTPPT,0.22,0.43,...,0,2364.12320,3,788.71259,1.90,67.74,5160,20110105_LM_HJ_Epac_Rev_007_F9.raw,T,435
6561,Q9Y6I3,1,Epsin-1 OS=Homo sapiens GN=EPN1 PE=1 SV=1 - [E...,Q9Y6I3,T439,Epsin-1 (EH domain-binding mitotic phosphoprot...,EPN1,PAATPTPTPPTRKTP,0.22,0.43,...,0,2364.12320,3,788.71259,1.90,67.74,5160,20110105_LM_HJ_Epac_Rev_007_F9.raw,T,439
6564,Q9Y6K1,1,DNA (cytosine-5)-methyltransferase 3A OS=Homo ...,Q9Y6K1,S105,DNA (cytosine-5)-methyltransferase 3A (Dnmt3a)...,DNMT3A,EPQPEEGSPAGGQKG,1.00,1.12,...,1,1789.81108,2,895.40918,-0.36,26.93,2058,20110105_LM_HJ_Epac_Rev_007_F29_2.raw,S,105
6578,Q9Y6R0,1,Numb-like protein OS=Homo sapiens GN=NUMBL PE=...,Q9Y6R0,S224,Numb-like protein (Numb-related protein) (Numb-R),NUMBL,TSFAREGSFRLSGGG,0.82,0.96,...,1,1683.79465,3,561.93640,-1.01,65.83,4199,20110105_LM_HJ_Epac_Rev_007_F33.raw,S,224


In [8]:
def match_seq_to_genename(dataset, seq_column):
    '''
    Maps amino acid sequences to gene names using the loaded fasta file.
    
    args:
    =====
    dataset: <pd.Dataframe> with a column of amino acid sequences
    seq_column: <str> column name containing amino acid sequences
    
    out:
    ====
    dataset: <pd.Dataframe> with an additional column containing gene names
    '''    

    fasta_sequence = list(SeqIO.parse(open(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/UP000005640_9606.fasta'), "fasta"))
    
    
    gene_dict = {}
    
    # iterate over rows in seq_column
    for i in dataset[seq_column]:
        print(i)
        i_str = str(i)
        for seq_record in fasta_sequence:
            matches = re.findall(re.escape(i_str), str(seq_record.seq))
            if matches:
                print(f"Match found for sequence: {seq_record}")
                gene_name_match = re.search(r"GN=(\w+)", seq_record.description)
                print('Gene name match:', gene_name_match)
                # gene_name_match = re.search("GN=(\w+)", seq_record.description)
                if gene_name_match:
                    gene_name = gene_name_match.group(1)
                    gene_dict[i] = gene_name
                    print(f"Match found: {i_str} -> {gene_name}")
                else: 
                    print(f"No gene name found in description for sequence: {i_str}")
    
    # map sequences to gene names           
    dataset['GeneName'] = dataset[seq_column].map(gene_dict) 
    print('Amino acid sequences matched to gene names.')
    return dataset 

In [9]:
data = match_seq_to_genename(data, 'Phosposite Sequence Context')

LATSESKSMPVLGSV
Match found for sequence: ID: sp|A0MZ66|SHOT1_HUMAN
Name: sp|A0MZ66|SHOT1_HUMAN
Description: sp|A0MZ66|SHOT1_HUMAN Shootin-1 OS=Homo sapiens OX=9606 GN=SHTN1 PE=1 SV=4
Number of features: 0
Seq('MNSSDEEKQLQLITSLKEQAIGEYEDLRAENQKTKEKCDKIRQERDEAVKKLEE...SNC')
Gene name match: <re.Match object; span=(56, 64), match='GN=SHTN1'>
Match found: LATSESKSMPVLGSV -> SHTN1
TECRRASSPSLIERN
Match found for sequence: ID: sp|A1L170|CA226_HUMAN
Name: sp|A1L170|CA226_HUMAN
Description: sp|A1L170|CA226_HUMAN Uncharacterized protein C1orf226 OS=Homo sapiens OX=9606 GN=C1orf226 PE=1 SV=1
Number of features: 0
Seq('MFENLNTALTPKLQASRSFPHLSKPVAPGSAPLGSGEPGGPGLWVGSSQHLKNL...SFE')
Gene name match: <re.Match object; span=(79, 90), match='GN=C1orf226'>
Match found: TECRRASSPSLIERN -> C1orf226
RLLPDPGSPRLAQNA
Match found for sequence: ID: sp|A6ND36|FA83G_HUMAN
Name: sp|A6ND36|FA83G_HUMAN
Description: sp|A6ND36|FA83G_HUMAN Protein FAM83G OS=Homo sapiens OX=9606 GN=FAM83G PE=1 SV=2
Number of features

In [10]:
data['Phosphosite'] = data['Amino acid'].astype(str) + '(' + data['Position'].astype(str) + ')'


In [11]:
keepcols = ['Phosphosite'] + ['GeneName'] + [col for col in data.columns if 'Log2 Ratio' in col]
data = data[keepcols]
data

,Phosphosite,GeneName,Log2 Ratio (Medium/Light) Forward,Log2 Ratio (Light/Medium) Reverse,Normalized Log2 Ratio (Medium/Light) Forward,Normalized Log2 Ratio (Light/Medium) Reverse
2,S(506),SHTN1,0.41,0.49,0.42,0.33
4,S(223),C1orf226,0.08,-0.03,0.10,-0.19
11,S(760),FAM83G,-1.18,-0.79,-1.16,-0.95
12,S(112),ZNF316,0.04,0.04,0.06,-0.12
14,S(246),CCDC85C,0.05,0.30,0.07,0.14
...,...,...,...,...,...,...
6560,T(435),EPN1,-2.19,-1.22,-2.17,-1.38
6561,T(439),EPN1,-2.19,-1.22,-2.17,-1.38
6564,S(105),DNMT3A,0.00,0.17,0.02,0.01
6578,S(224),NUMBL,-0.28,-0.06,-0.26,-0.22


In [12]:
def create_phos_ID(dataset):
    '''
    Concatenates GeneName and Phosphosite columns.
    
    args:
    =====
    dataset: <pd.Dataframe> with columns 'GeneName' and 'Phosphosite'
    
    out:
    ====
    dataset: <pd.Dataframe> with 'phosphosite_ID' column and 'GeneName' + 'Phosphosite' columns dropped
    '''
    dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)
    dataset = dataset.drop(columns=['Phosphosite', 'GeneName'])
    print('Phosphosite IDs created.')
    return dataset

data = create_phos_ID(data) # call function to create phosphosite_ID column

print('Phosphosite IDs created.')

Phosphosite IDs created.
Phosphosite IDs created.


/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_10876/3553972964.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)


In [13]:
def clean_phosID_col(data):
    data = data[~data.phosphosite_ID.str.contains('nan', case=False, na=False)]
    data = data[~data.phosphosite_ID.str.contains(';', case=False, na=False)]
    data = data[~data.phosphosite_ID.str.contains('-', case=False, na=False)]

    # Add this line to remove decimals from phosphosite_ID (e.g., S123.0 -> S123)
    data['phosphosite_ID'] = data['phosphosite_ID'].apply(lambda x: re.sub(r'\((\d+)\.0+\)', r'(\1)', x))
    
    data_grouped = data.groupby(by='phosphosite_ID')
    
    if len(data) != len(data_grouped):
        numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
        non_numeric_cols = data.columns.difference(numeric_cols + ['phosphosite_ID']).tolist()
        data_numeric = data_grouped[numeric_cols].mean()
        data_categorical = data_grouped[non_numeric_cols].first().reset_index()
        
        # Merge numeric and non-numeric parts
        data = pd.merge(data_categorical, data_numeric, on='phosphosite_ID')
        print('Phosphosites with multiple measurements have been averaged')
    else:
        print('There are no phosphosites with multiple measurements')

    # Replace inf values with NaNs
    data = data.replace([np.inf, -np.inf], np.nan)
    
    # Ensure phosphosite_ID is first column
    if data.columns[0] != 'phosphosite_ID':
        phosphosite_ID = data.pop('phosphosite_ID')
        data.insert(0, 'phosphosite_ID', phosphosite_ID)

    return data

In [14]:
data = clean_phosID_col(data)
print("After cleaning phosphosite_ID column:")
data

There are no phosphosites with multiple measurements
After cleaning phosphosite_ID column:


,phosphosite_ID,Log2 Ratio (Medium/Light) Forward,Log2 Ratio (Light/Medium) Reverse,Normalized Log2 Ratio (Medium/Light) Forward,Normalized Log2 Ratio (Light/Medium) Reverse
2,SHTN1_S(506),0.41,0.49,0.42,0.33
4,C1orf226_S(223),0.08,-0.03,0.10,-0.19
11,FAM83G_S(760),-1.18,-0.79,-1.16,-0.95
12,ZNF316_S(112),0.04,0.04,0.06,-0.12
14,CCDC85C_S(246),0.05,0.30,0.07,0.14
...,...,...,...,...,...
6560,EPN1_T(435),-2.19,-1.22,-2.17,-1.38
6561,EPN1_T(439),-2.19,-1.22,-2.17,-1.38
6564,DNMT3A_S(105),0.00,0.17,0.02,0.01
6578,NUMBL_S(224),-0.28,-0.06,-0.26,-0.22


In [15]:
data.to_csv(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/PreprocessedDatasets/LATM2013.csv', index=False)


print(dataset, 'has been saved to CSV successfully!', data)

LATM2013 has been saved to CSV successfully!        phosphosite_ID  Log2 Ratio (Medium/Light) Forward  \
2        SHTN1_S(506)                               0.41   
4     C1orf226_S(223)                               0.08   
11      FAM83G_S(760)                              -1.18   
12      ZNF316_S(112)                               0.04   
14     CCDC85C_S(246)                               0.05   
...               ...                                ...   
6560      EPN1_T(435)                              -2.19   
6561      EPN1_T(439)                              -2.19   
6564    DNMT3A_S(105)                               0.00   
6578     NUMBL_S(224)                              -0.28   
6579     NUMBL_S(263)                               0.12   

      Log2 Ratio (Light/Medium) Reverse  \
2                                  0.49   
4                                 -0.03   
11                                -0.79   
12                                 0.04   
14                 